In [1]:
from shapely.geometry import box
import matplotlib.pyplot as plt
from contextily import add_basemap
import contextily as cx
from shapely.geometry import Point, Polygon
from shapely.ops import transform
from pyproj import Transformer
import geopandas as gpd
import numpy as np
import time
import requests
import pandas as pd
from shapely import wkt

In [4]:
potention_gdf = gpd.read_file('Seattle_High_Priority_Park_Development_Area.shp')
potention_gdf.head()

,title,SymbolID,SHAPE_Leng,SHAPE_Area,geometry
0,Polygon,Override 1,14539.945026,5.225315e+06,"POLYGON ((-13619441.269 6062711.767, -13619401..."
1,Polygon,Override 1,7322.903446,2.343306e+06,"POLYGON ((-13617137.81 6062238.81, -13617147.3..."
2,Polygon,Override 1,18258.617626,5.602645e+06,"POLYGON ((-13613834.592 6060836.652, -13613837..."
3,Polygon,Override 1,3043.333562,4.164205e+05,"POLYGON ((-13618270.034 6059174.163, -13619358..."
4,Polygon,Override 1,3000.647972,5.344401e+05,"POLYGON ((-13614014.611 6058312.902, -13614381..."


In [5]:
# 添加唯一标识
potention_gdf['SymboID'] = potention_gdf.index

In [6]:
potention_gdf.head()

,title,SymbolID,SHAPE_Leng,SHAPE_Area,geometry,SymboID
0,Polygon,Override 1,14539.945026,5.225315e+06,"POLYGON ((-13619441.269 6062711.767, -13619401...",0
1,Polygon,Override 1,7322.903446,2.343306e+06,"POLYGON ((-13617137.81 6062238.81, -13617147.3...",1
2,Polygon,Override 1,18258.617626,5.602645e+06,"POLYGON ((-13613834.592 6060836.652, -13613837...",2
3,Polygon,Override 1,3043.333562,4.164205e+05,"POLYGON ((-13618270.034 6059174.163, -13619358...",3
4,Polygon,Override 1,3000.647972,5.344401e+05,"POLYGON ((-13614014.611 6058312.902, -13614381...",4


In [7]:
potention_gdf.set_crs(epsg = '3857')
potention_gdf = potention_gdf.to_crs(epsg = '3857')

In [9]:
grid_data = pd.read_csv('park_info_with_grids.csv')
# csv比geojson更好,存储和阅读出错率更小,不占内存
# 不要忘记在打开和存储csv时要用wkt转换geometry格式
grid_data.head()

,Unnamed: 0,geometry,grid_id,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count,job_count,nearest_residential_distance,...,nearest_park_distance,distance_to_nearest_bike_network,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,TOTAL_AREA,TYPE
0,0,"POLYGON ((-13607990.19116259 6023951.23868917,...",0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,1,POLYGON ((-13608506.81627558 6023962.862982571...,1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2,"POLYGON ((-13608682.76518992 6023966.61165225,...",2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,3,POLYGON ((-13610206.44615618 6023341.900384837...,3,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,4,POLYGON ((-13608853.09858667 6024384.237879843...,4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [10]:
from shapely import wkt
grid_data['geometry'] = grid_data['geometry'].apply(wkt.loads)

In [11]:
grid_data_gdf = gpd.GeoDataFrame(grid_data,geometry = 'geometry')

In [12]:
grid_data_gdf.head()

,Unnamed: 0,geometry,grid_id,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count,job_count,nearest_residential_distance,...,nearest_park_distance,distance_to_nearest_bike_network,POP_DENSITY,EMPLOYED_PERCENTAGE,UNDER5_PERCENTAGE,65_PERCENTAGE,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,TOTAL_AREA,TYPE
0,0,"POLYGON ((-13607990.191 6023951.239, -13608158...",0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,1,"POLYGON ((-13608506.816 6023962.863, -13608549...",1,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,2,"POLYGON ((-13608682.765 6023966.612, -13608549...",2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,3,"POLYGON ((-13610206.446 6023341.9, -13610330.2...",3,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,4,"POLYGON ((-13608853.099 6024384.238, -13608765...",4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [14]:
grid_data_gdf = grid_data_gdf.set_crs(epsg = '3857')
grid_data_gdf = grid_data_gdf.to_crs(epsg = '3857')

In [17]:
# 空间连接
intersected_gdf_grids = gpd.sjoin(grid_data_gdf,potention_gdf,how = 'inner',predicate = 'intersects')
intersected_gdf_grids.head()

,Unnamed: 0,geometry,grid_id,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count,job_count,nearest_residential_distance,...,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,TOTAL_AREA,TYPE,index_right,title,SymbolID,SHAPE_Leng,SHAPE_Area,SymboID
64,64,"POLYGON ((-13610060.023 6026788.165, -13610451...",64,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,14,Polygon,Override 1,37539.785477,3.725596e+07,14
66,66,"POLYGON ((-13609668.636 6027004.685, -13610060...",66,0.0,2.0,86.0,56.0,19.0,0.0,48.909772,...,24.665744,7.558706,3948.0,4.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
68,68,"POLYGON ((-13610451.392 6026571.635, -13610842...",68,5.0,1.0,464.0,106.0,97.0,0.0,378.796898,...,36.033441,7.823034,2829.0,2.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
76,76,"POLYGON ((-13610667.273 6026964.204, -13611058...",76,2.0,0.0,274.0,135.0,35.0,0.0,75.421703,...,46.413197,10.016453,2030.0,2.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
77,77,"POLYGON ((-13609940.734 6027499.573, -13609884...",77,0.0,0.0,0.0,0.0,0.0,0.0,266.222009,...,42.979452,0.513699,421.0,1.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14


In [18]:
intersected_gdf_grids.groupby('grid_id').agg({
    'Schools_count': 'sum',
    'Landmarks_count': 'sum',
    'Metros_count': 'sum',
    'Housings_count': 'sum',
    'Affordable_Housings_count': 'sum',
    'job_count': 'sum',
    'nearest_residential_distance': 'mean',
    'nearest_MHA_distance': 'mean',
    'nearest_metro_distance': 'mean',
    'nearest_school_distance': 'mean',
    'nearest_landmark_distance': 'mean',
    'nearest_park_distance': 'mean',
    'distance_to_nearest_bike_network': 'mean',
    'POP_DENSITY': 'mean',
    'EMPLOYED_PERCENTAGE': 'mean',
    'UNDER5_PERCENTAGE': 'mean',
    '65_PERCENTAGE': 'mean',
    'PEOPLE_OF_COLOR_PERCENTAGE': 'mean',
    'POVERTY_PERCENTAGE': 'mean',
    'TOTAL_AREA': 'sum',
    'TYPE': 'first'
    # 添加更多你想要计算的列
}).reset_index()
intersected_gdf_grids

,Unnamed: 0,geometry,grid_id,Schools_count,Landmarks_count,Metros_count,Housings_count,Affordable_Housings_count,job_count,nearest_residential_distance,...,PEOPLE_OF_COLOR_PERCENTAGE,POVERTY_PERCENTAGE,TOTAL_AREA,TYPE,index_right,title,SymbolID,SHAPE_Leng,SHAPE_Area,SymboID
64,64,"POLYGON ((-13610060.023 6026788.165, -13610451...",64,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,14,Polygon,Override 1,37539.785477,3.725596e+07,14
66,66,"POLYGON ((-13609668.636 6027004.685, -13610060...",66,0.0,2.0,86.0,56.0,19.0,0.0,48.909772,...,24.665744,7.558706,3948.0,4.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
68,68,"POLYGON ((-13610451.392 6026571.635, -13610842...",68,5.0,1.0,464.0,106.0,97.0,0.0,378.796898,...,36.033441,7.823034,2829.0,2.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
76,76,"POLYGON ((-13610667.273 6026964.204, -13611058...",76,2.0,0.0,274.0,135.0,35.0,0.0,75.421703,...,46.413197,10.016453,2030.0,2.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
77,77,"POLYGON ((-13609940.734 6027499.573, -13609884...",77,0.0,0.0,0.0,0.0,0.0,0.0,266.222009,...,42.979452,0.513699,421.0,1.0,14,Polygon,Override 1,37539.785477,3.725596e+07,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,2659,"POLYGON ((-13619619.181 6062736.469, -13619535...",2659,0.0,0.0,0.0,0.0,0.0,0.0,539.465735,...,36.073826,1.062640,4450.0,1.0,0,Polygon,Override 1,14539.945026,5.225315e+06,0
2661,2661,"POLYGON ((-13619262.299 6062736.633, -13619535...",2661,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,0,Polygon,Override 1,14539.945026,5.225315e+06,0
2661,2661,"POLYGON ((-13619262.299 6062736.633, -13619535...",2661,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,1,Polygon,Override 1,7322.903446,2.343306e+06,1
2664,2664,"POLYGON ((-13620644.678 6062737.048, -13620537...",2664,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,0.0,NaN,0,Polygon,Override 1,14539.945026,5.225315e+06,0


In [19]:
newpark = intersected_gdf_grids

In [ ]:
newpark.to_csv('new_park.csv')